In [13]:
import os
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Function to get the list of image files
def get_image_files(test_dir):
    image_files = []

    # Iterate over each species directory
    for species_dir in os.listdir(test_dir):
        species_path = os.path.join(test_dir, species_dir)
        
        # Check if it's a directory
        if os.path.isdir(species_path):
            # Iterate over each audio_file directory
            for audio_dir in os.listdir(species_path):
                audio_path = os.path.join(species_path, audio_dir)
                
                # Check if it's a directory
                if os.path.isdir(audio_path):
                    # Iterate over each image file in the audio_file directory
                    for file in os.listdir(audio_path):
                        if file.endswith(('png', 'jpg', 'jpeg')):
                            image_files.append(os.path.join(audio_path, file))
    
    return image_files

# Define the directory structure
test_dir = '../test/'

# Get the list of image files
image_files = get_image_files(test_dir)

# Print the list of image files
print("List of image files in the test directory:")
for file in image_files:
    print(file)

# Continue with the rest of the code for model evaluation
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_batches = test_datagen.flow_from_directory(test_dir,
                                                target_size=(224, 224),
                                                class_mode='categorical',
                                                shuffle=False,
                                                batch_size=4)

# show class indices
print('****************')
for cls, idx in test_batches.class_indices.items():
    print('Class nr ', idx, ' -> ', cls)
print('****************')

net = MobileNetV2(include_top=False,
                  weights='imagenet',
                  input_tensor=None,
                  input_shape=(224, 224, 3))
x = net.output
x = Flatten()(x)
x = Dropout(0.5)(x)
output_layer = Dense(5, activation='softmax', name='softmax')(x)
loaded_model = Model(inputs=net.input, outputs=output_layer)

# load weights into the new model
loaded_model.load_weights("models/AM_mobilenetV2.h5")
print("Loaded model from disk")

# evaluate the loaded model on the test data
loaded_model.compile(optimizer=Adam(lr=5e-5),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

score = loaded_model.evaluate(test_batches, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1] * 100))


ModuleNotFoundError: No module named 'tensorflow.python.keras.applications'

In [12]:
import os

test_dir = '../data/test/'
image_files = [os.path.join(test_dir, file) for file in os.listdir(test_dir) if file.endswith(('png', 'jpg', 'jpeg'))]

print("List of image files in the test directory:")
for file in image_files:
    print(file)


List of image files in the test directory:
